In [4]:
# The goal with this training exercise will be to produce visualizations of real-time COVID-19
# data that is being provided by usafacts.org (which is where the CDC
# gets their numbers)

# the structure of the program will be to 
# 1.  Read in and filter the data from the online repository
#  1a.  make a plot of the COVID cases per county for several different counties
# to show that un-normalized data can be misleading
# 2.  Calculate rate of increase per day, normalized to the county; and compute a running average of cases
# 2a.  make plots that take a random draw of counties and plots their normalized
#  cases over time
# 3a.  Compute higher order time derivatives of the plots to make a predictive model

# throughout the program will be comment parts the begin with "#advanced programming:" 
# these are ways to augment the program beyond its current capabilities; 
# as you progress through your training this summer, I encourage you to try and tackle 
# some of these advanced programming tasks to better your own knowledge

# we are going to use the pandas package to read in and store the data
# pandas is useful for when the data in your arrays is not entirely numerical
# you can go to the webURL https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/
# to see where we're obtaining the data for this program

%reset -f   

            # this is a reset command that resets all of your variables and values
           # it's a good habit to put this at the beginning of all of your programs 
            # for when you need to run it from scratch; it effectively does the same thing 
            # as resetting the kernel but kernel resets should be used more for debugging
#restart()

In [5]:
# this cell contains all of my library imports; whether I use all of these libraries doesn't matter
# I'm putting here anything that I think I *might* use
# you can think of this first cell as your toolbox for solving the problem of this program
import csv # csv is a library that allows python to recognized Comma Separated Value files, which 
            # are arrays that store data in a particularly common format that we will be reading
import pandas as pd  # pandas is a special package that is designed to store and manipulate data
                    # in arrays very quickly and efficiently
import numpy as np  # good ol' numpy for all its array goodness
import datetime   # this is a special function that will allow us to translate the dates in the file, 
                # which are normally just read as numbers, into dates that can be read and understood
                # by the progrma as times
import matplotlib.pyplot as plt   # for access to plotting tools

pd.set_option('display.min_rows', 15) # this is an option I'm setting to control the output rows we display 
pd.set_option('display.max_rows', 30) # in the notebook; since the dataset is much larger than we should ever 
                    # display all at once, we can control how many rows are output with a command like this
                            

In [6]:
# this cell reads in COVID-19 data from a URL
# you should go to this URL to see what the data looks like before reading it into your program

# when reading in data from a table using pandas, there are special read functions
# to handle files that are in various formats;  the format of the data we're looking at is csv
# which is the default output from excel spreadsheets; csv stands for "comma separated values"
# meaning that every element in the array is separated by a comma

# by default, the pandas function "read_csv" will assume that the first line in the file 
# is the header information from the columns in the data file, the header contains information 
# we want as header info, but it also contains dates which are actual data we want to store 
# and work with, not just keep as header information 

# since the header contains data we need, and the default for the read function expects a header 
# row, then we need to tell the read runction not to read it in as a normal header, so we 
# set the option (header=None) in the function argument, after the URL

# each object that is created below (cases, cases_h, pop, and deaths) can be thought of as an array, but it
# is more precisely a PANDAS DATAFRAME; a dataframe is like an array with extra info that pandas
# uses to speed up all operations we do with that array; so, in this example, I will refer
# to our data array as a dataframe, but for visualizing them in your mind, they are effectively
# the same thing as a normal array

# here i'm reading in the data without the header
# this file contains the number of cases over time separated by county, which is the data
# I'm going to start with

# here is the function call that grabs the data without the header (keyword argument header=None)
cases = pd.read_csv(
    'https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv',
    header=None)

# here i'm reading in the data with the header; though we will be working primarily with the cases dataframe 
# data, which does not contain the formatted header info we will also be using cases_h at some point 
# to take advantage of how Pandas stores info (you'll see why I'm doing this later); since I'm reading 
# the header info, and some of that information is dates, I'm putting in a special option so that it 
# doesn't think those are just numbers (parse_dates=True)
cases_h = pd.read_csv(
    'https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv',
    parse_dates=True)

# the above function calls will get the most recent data from that site.  if the site url or data format
# changes, then the above call will have to be changed

# pop is for reading in the population of each county; uncomment this function call when you get to the point
# in the program where it tells you to compute per capita data
#pop = pd.read_csv(
#    'https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv')

# deaths is for reading in the confirmed covid deaths; uncomment this function call when you're ready to 
# to face the reaper and do some death analysis
#deaths = pd.read_csv(
#    'https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_deaths_usafacts.csv')

np.shape(cases) # here is the shape of the dataframe that the data created; the format is rows, columns
                # there should be around 3196 rows (1 for each county in the US), and some number of columns
                # in the hundreds, which are the number of days since data collection began

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [ ]:
np.shape(cases_h)   # notice that cases_h, the one that read the header and kept it in the header, rather 
                    # than moving the header info into the array, has one fewer row than the cases dataframe

In [ ]:
# a summary output of how pandas sees the data stored in cases
# the data that's contained in this file is the cumulative number of cases reported
# each day in that county
cases

#cases_h
#uncomment the above line to see what cases_h looks like with the header info; be sure you can see the difference
# between cases and cases_h

In [ ]:
# Since each row is a different county, I can access data for a specific county 
# by selecting a full row; in a pandas dataframe, a full row is not just the index of the row, it is a range
# writing cases[108:109] will show me all of index 108 for this pandas dataframe, 
# which is Maricopa County in Arizona
#cases[108:114]  # if you want to print a range of cases

temp = cases_h[cases_h['County Name']=='Maricopa County ']  # if 
temp
#temp
#cases[107:108]

In [ ]:
# the first 4 rows in our transposed dataframe are just header information
# the data to be plotted is all contained after row 4 and can be selected by county

# for all of the data I'm going to plot, I'll need the same range of dates for each
# so here's how I isolate those into a variable I'm calling dates


#dates = t_cases[0][4:]; 
dates = pd.to_datetime(t_cases[0][4:])  # a couple of things here:  "to_datetime" is a pandas 
                    # function that converts the input
                    # into a time-like object that pandas can easily read/manipulate;  t_cases[0][4:] is a special 
                    # way of formatting an array call (called a 'slice') so that it only displays a subset of the 
                    # data; in this case, I'm selecting [0] for the first index, 
                    # meaning the first column in the dataframe t_cases, 
                    # and [4:] to say that I want to start with the 4th index (meaning the fifth row) and then 
                    # read all of those values as dates; 
                    # so in summary, I'm just picking out the dates from the array


t_cases[0][4:]  # here is what the subset of the dataframe looks like with the selection [0][4:] that I made above

#to select a specific column x, we use t_cases[:][x]; so for Maricopa 
#t_cases[:][107]

# if we want it without the header info, use t_cases[4:][108]
#t_cases[4:][107]


# advanced programming:  go through and edit above this cell to request and store a user selected list of counties
# to compare against one another

In [ ]:
# and here is what pandas sees now that we've told it to interpret those as dates
# notice that the information is the same, but the format has changed
# also, the datatype (dtype) has now changed from an 'object' to 'datetime64', which is the special 
# time format that pandas likes
dates

In [ ]:
# Now, our goal with this program is to use the larger dataset in t_cases to make plots 
# of individual counties over time.  
# initially, we'd like to take an arbitrary number of counties that we define, compute the per-capita total 
# COVID cases, and plot those all together over the same time range 

# advanced programming:  take a user defined set of counties, compute and plot the 2,5, and 9 day running 
# averages over the same time range, along with their associated errors


# So, if we can write our program to extract and plot 1 arbitrary county from the dataframe
# then we just iterate over some list of counties in order to produce a plot with 
# multiple counties

# In order to do this with Pandas, we have to recognize that the format the 
# data was read in, is not the format Pandas prefers.
# Pandas and all its time functionality seem built around the idea of 
# the time series being all in one column

# our case was further complicated by the fact that some of the data (namely the dates), 
# are being stored in the header, rather than in the frame itself
# to get around this, we've used the read-in method to put the header information into the dataframe itself, 
# rather than just in the header, and transposed that dataframe so that the dates are now in the first column

# in extracting the data for a county, it will ultimately be easier to reorganize, 
# or recreate our dataframe so that it only has a "date" column, and a value column

# So, let's try to create a new data frame here
# it should have dates as rows in the first column
# it should have county_FIPS (a unique  identifier for each county) as columns in the first row
# it should have the values for each county and each date in the rest of the array

In [ ]:
# Making a new data frame

# we created dates above, which will serve as one column in our new dataframe

# for the other columns, which show each counties numbers, 
# we want all but the first (since the first column contained the dates)
# we can do this by using the .loc and .iloc functions, which are two very fast methods
# for sifting through data; any dataframe can use the loc functions (like t_cases here)
# the difference between .loc and .iloc is that .loc is intended to be used for column/label data
# whereas .iloc is used for integer values within the dataframe

# the main reason that they are different is that pandas is a specifically programmed package to do things
# with dataframes very quickly, and especially so for specific datatypes within that dataframe
# since our # of cases, population, and deaths data are all integers, it makes sense to 
# utilize this extra efficient method provided by pandas

# as an argument loc wants to know what data within the frame you'd like to select.  it then stores the location
# of the data, rather than the data itself, so that anytime I want to use that data
# I can just point to it with the variable t_cases_cols defined below

# type t_cases.columns[1:] in a separate execution cell to see what kind of object it produces

#t_cases_cols = t_cases.columns[1:]

t_cases_cols = t_cases.loc[0,1:40000] # here, my values in brackets are stating what 
                        # data I want.  The first coordinate, 0, says to start with the first row
                        # (which is where all of the county IDs are stored)
                        # and go from the second column all the way to the end 
                        #  going over list of county IDs from beinning to end,
                        #  the numbering is not linear. the county IDs go up to 56037
                        # but there are only about 3200 counties total
#t_cases_cols = t_cases[0]
#t_cases_cols

# so we've got our dates as rows, our counties as columns, and now we want the array to be 
# populated with the data for each date and county
#t_cases_cols
# to accomplish this, I'm going to use a function called iloc, which like .loc above, stores only the 
# positions of the data in the dataframe, without moving, or modifying the values themselves.  
# accidently modifying array values is a common 
# issue that happens with beginner programmers, and so using iloc help to prevent that from occuring

cts = t_cases.iloc[4:,1:]; # in the function call for iloc, it wants a range of rows and columns 
                            # from which to store the location info; so I've chosen [4:,1:] to select 
                            # the data that starts in the 5th row (index [4:]), and the second column
                            # (index [1:])
                            # it's important to note here that this method of slicing creates a new 
                            # dataframe from t_cases.  Since t_cases is a dataframe itself
                            # I found that I needed to recast it later as a 2D numpy array 
                            # before then recombining it with the other info to make my new dataframe
            

# with all 3 of these variables (dates, t_cases.columns, and cts)
# I can now reassemble my data into a pandas dataframe


In [ ]:
# need to check data type of the list we just created
cts.dtypes 

In [ ]:
# it says the data are all type 'object', which is sort-of a pandas catch all data type
# if we want to be able to work with the data in any way, it's preferable to be more specific
# as to what the data type really is; in this instance, number of cases is an intenger value
# so we should change the dataype in the cts thusly: 
cts = cts.astype(int)
cts.dtypes
#cts

In [ ]:
# since I'm going to be creating a data frame, I need to ensure that all of the dimensions match up
# in particular, I want the shape of dates to be all rows, the shape of t_cases_cols to be all columns, 
# and the shape of cts to have "dates" number of rows, and "t_cases_cols" number of columns

dates.shape, t_cases_cols.shape, cts.shape, 

In [ ]:
# the data types don't need to match up, but it's sometimes helpful to know what they are, since pandas
# treats different data types differently in its operations 
# if you want to check the datatypes, you can uncomment the below statement
#print(dates.dtypes,"\n\n",cts.dtypes,"\n\n",t_cases_cols)  # the "\n" is how you force it to print a new line

In [ ]:
# here is where I recast my dataframe cts, as a numpy array, which I'm then going to stick into my custom-made
# data frame for plotting
# cast data frame as numpy array
ncts= cts.to_numpy()